In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By 

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
import time

import pandas as pd

# Create EdgeOptions instance
edge_options = Options()
edge_options.add_argument("--guest")

# Initialize WebDriver with options
driver = webdriver.Edge(options=edge_options)
driver.maximize_window()

# Open Google Jobs
search_keyword = "data science jobs"
driver.get(f"https://www.google.com/search?q={search_keyword}&ibp=htl;jobs")
time.sleep(3)

In [ ]:
# Create a df to populate
df = pd.DataFrame(columns=['Job Posting', 'Company', 'Location', 'URL'])

keep_scrolling = True

seen_postings = set()

while keep_scrolling:
    all_items = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "PwjeAc"))
    )

    new_postings_found = False

    job_posting_elements = []
    for item in all_items:
        job_posting_elements.append(item)

    for item in job_posting_elements:
        try:
            position_element = item.find_element(By.CLASS_NAME, "BjJfJf.PUpOsf")
            position_text = position_element.text.strip()

            company_element = item.find_element(By.CLASS_NAME, "vNEEBe")
            company_text = company_element.text.strip()

            location_element = item.find_element(By.CLASS_NAME, "Qk80Jf")
            location_text = location_element.text.strip()

            posting_key = (position_text, company_text, location_text)

            if posting_key not in seen_postings:
                seen_postings.add(posting_key)
                new_postings_found = True

                print(f"Job Position: {position_text}")
                print(f"Company: {company_text}")
                print(f"Location: {location_text}")

                try:
                    link_element = item.find_element(By.CSS_SELECTOR, "a.pMhGee")
                    link_url = link_element.get_attribute("href")
                except NoSuchElementException:
                    link_url = "N/A"

                df = df.append({'Job Posting': position_text, 'Company': company_text, 'Location': location_text, 'URL': link_url}, ignore_index=True)

        except (StaleElementReferenceException, NoSuchElementException):
            continue

    if not new_postings_found:
        keep_scrolling = False
    else:
        # Scroll the side menu
        side_menu = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "zxU94d.gws-plugins-horizon-jobs__tl-lvc"))
        )
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", side_menu)
        time.sleep(15)  # wait to load more posts


In [ ]:
df.to_excel("results.xlsx", index = False)

In [ ]:
len(df)  # confirm num of entries for tracking